# Amazon Rekognition Demo

***
Copyright [2017]-[2017] Amazon.com, Inc. or its affiliates. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. A copy of the License is located at

http://aws.amazon.com/apache2.0/

or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
***

**Prerequisites:**

The user or role that executes the commands must have permissions in AWS Identity and Access Management (IAM) to perform those actions. AWS provides a set of managed policies that help you get started quickly. For our example, you need to apply the following minimum managed policies to your user or role:

* AmazonRekognitionFullAccess
* AmazonDynamoDBFullAccess
* AmazonS3FullAccess

Be aware that we recommend you follow AWS IAM best practices for production implementations, which is out of scope fof this workshop.



In [ ]:
# initialise Notebook
import boto3
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
from pprint import pprint
from io import BytesIO
import os
import io

# key variable initiation
rekognition = boto3.client('rekognition', region_name='eu-west-1')
dynamodb = boto3.resource('dynamodb', region_name='eu-west-1')
s3 = boto3.client('s3')

# blue, green, red, grey
colors = ((220,220,220),(242,168,73),(76,182,252),(52,194,123))

# helper function to lookup names in dynamoDB
actors = dynamodb.Table('rekognition_actors')
def get_actor_name(actorId):
    response = actors.get_item(
            Key={
                'actorId': actorId
            },
            AttributesToGet=['actorFullName']                  
        )
    
    if 'Item' in response:
        return (response['Item']['actorFullName'])
    else:
        return ('no match found in actors lookup')

In [ ]:
%%bash
aws dynamodb create-table --table-name rekognition_actors \
--attribute-definitions AttributeName=actorId,AttributeType=S \
--key-schema AttributeName=actorId,KeyType=HASH \
--provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1 \
--region eu-west-1

## Object and scene detection

This features allows you to detect thousands of objects, scenes and concepts in your images and provide labels describing the objects detected.

This example presents an image to the object and scene detection endpoint to determine the lables for the image. 

In [ ]:
!mkdir tmp

In [ ]:
bucket = 'mlrekognitiondemo'
key = 'images/P1010138.JPG'
#key = 'images/P4050012.JPG'

# retrieve labels from Rekognition service
ret = rekognition.detect_labels(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name" : key,
        }
    },
    # MaxLabels=5,
    # MinConfidence=85
)

# Output image and labels
image_location = 'tmp/image'
s3.download_file(bucket, key, image_location)
size = 500,500
image_bin=Image.open(image_location)
image_bin.thumbnail(size, Image.ANTIALIAS)

display(image_bin)

pprint(ret['Labels'])


## Facial analysis

Using Amazon DetectFaces API, you can detect faces in an image and key facial characteristics.

As faces are detected on an image you get a number of attributes about the face:

	- gender
	- age
	- sentiment of the person
    - coordinates of key features
    - image quality
    - etc.


In [ ]:
bucket = 'mlrekognitiondemo'
key = 'images/Werner_Vogels.jpeg'

# detect faces using Rekognition service
ret = rekognition.detect_faces(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name" : key,
        }
    },
    Attributes=['ALL'],
)

# Output labels

pprint (ret['FaceDetails'])


In [ ]:
# retrieve and resize image for display

image_location = 'tmp/image'
s3.download_file(bucket, key, image_location)
size = 400,400
image_bin=Image.open(image_location)
image_bin.thumbnail(size, Image.ANTIALIAS)

display(image_bin)

In [ ]:
# Detect multiple faces within images

bucket = 'mlrekognitiondemo'
key = 'images/faces.png'

# retrieve face boxes from Rekognition service

ret = rekognition.detect_faces(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name" : key,
        }
    },
    Attributes=['DEFAULT'],
)

boxes = []

# Output labels
faces = ret['FaceDetails']
for face in faces:
    pprint (face['BoundingBox'])
    boxes.append (face['BoundingBox'])
    

In [ ]:
# donwload image from S3
image_location ='tmp/image'
s3.download_file(bucket, key, image_location)

image_out = Image.open("tmp/image")

# apply face boxes to image 
draw = ImageDraw.Draw(image_out)
width, height = image_out.size
col = 0
line= 10
for box in boxes:
    x1 = int(box['Left'] * width)
    y1 = int(box['Top'] * height)
    x2 = int(box['Left'] * width + box['Width'] * width)
    y2 = int(box['Top'] * height + box['Height']  * height)
        
    for l in range(line):
        draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
    col += 1

size = 500,500
image_out.thumbnail(size, Image.ANTIALIAS)
   
display(image_out)


## Face comparison

Given two images with faces, Rekognition will compare the largest face from the source image and find similarity with faces found in the target image. 


In [ ]:
# retrieve and resize images for display

bucket = 'mlrekognitiondemo'
key1 = 'images/coty.jpg'
key2 = 'images/Werner_Vogels.jpeg'

s3.download_file(bucket, key1, 'tmp/image1')
s3.download_file(bucket, key2, 'tmp/image2')

display(HTML("<table><tr><td><img src='tmp/image2' height='250' width='250'></td><td><img src='tmp/image1' height='250' width='250'></td></tr></table>"))

In [ ]:
# Compare faces using Rekognition

ret = rekognition.compare_faces(
    SourceImage={
        "S3Object": {
            "Bucket": bucket,
            "Name" : key2,
        }
    
    },
    TargetImage={
        "S3Object": {
            "Bucket": bucket,
            "Name" : key1,
        }    
    }
)

# Output match(es)
pprint (ret)

In [ ]:
# And highlight match on the image

image_out = Image.open("tmp/image1")
draw = ImageDraw.Draw(image_out)

# get image size
width, height = image_out.size

col = 2
line= 5

box= ret['FaceMatches'][0]['Face']['BoundingBox']
x1 = int(box['Left'] * width)
y1 = int(box['Top'] * height)
x2 = int(box['Left'] * width + box['Width'] * width)
y2 = int(box['Top'] * height + box['Height']  * height)
        
for l in range(line):
        draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
    
        

display(image_out)

## Face recognition

With FaceRekognition, you can search your image collection for similar faces by storing faces, using the IndexFaces API operation. 
You can then use the SearchFaces operation to return high-confidence matches. A face collection is an index of faces that you own and manage.




In [ ]:
%%bash
aws rekognition create-collection --collection-id actors --region eu-west-1

In [ ]:
# load face definitions into index

faces = [
{"objectId":"images/z5oj3dys.jpg","actorFullName":"Alexandra Maria Lara"},
{"objectId":"images/z6od3gys.jpg","actorFullName":"Alexandra Maria Lara"},
{"objectId":"images/z7ad3fys.jpg","actorFullName":"Alexandra Maria Lara"},    
{"objectId":"images/z5kj4dss.png","actorFullName":"Alexandra Maria Lara"},   
{"objectId":"images/z6hj7fss.jpg","actorFullName":"Alexandra Maria Lara"}, 
{"objectId":"images/z6hd7hsl.jpg","actorFullName":"Alexandra Maria Lara"},      
{"objectId":"images/i3bvrk3c.jpg","actorFullName":"Matthias Schweighoefer"},
{"objectId":"images/idbvyk3c.png","actorFullName":"Matthias Schweighoefer"},
{"objectId":"images/i2bvrg7d.jpg","actorFullName":"Matthias Schweighoefer"},
{"objectId":"images/d5bgrg6d.jpg","actorFullName":"Matthias Schweighoefer"}, 
{"objectId":"images/d2sgrg8a.jpg","actorFullName":"Matthias Schweighoefer"},     
{"objectId":"images/hq1rszje.jpg","actorFullName":"Franz Hagn"},
{"objectId":"images/hq2r5yje.png","actorFullName":"Franz Hagn"},
{"objectId":"images/06e3ykz8.jpg","actorFullName":"Katrin Bauerfeind"}, 
{"objectId":"images/hgx13oet.jpg","actorFullName":"Tom Beck"},
{"objectId":"images/06e3gkf8.jpg","actorFullName":"Tony Garrn"},
{"objectId":"images/evd9sm37.png","actorFullName":"Karoline Herfurth"},
{"objectId":"images/evd2am47.png","actorFullName":"Karoline Herfurth"},  
{"objectId":"images/eva8sm27.jpg","actorFullName":"Karoline Herfurth"},    
]

s3_image_bucket = 'mlrekognitiondemo'
face_table = dynamodb.Table('rekognition_actors')

# function to write reference table in DynamoDB
def populate_lookup_table(actorFullName,actorId):
    print ('Populating DynamoDB index for ' + actorFullName + ' as ' + actorId)
    response = face_table.put_item(
                Item={
                      'actorId': actorId,
                      'actorFullName': actorFullName,
                      }
                )
    return response
        
def populate_rekognition_collection(objectId):
    print ('Indexing ', objectId)
    ret = rekognition.index_faces(
    CollectionId=rekognition_face_index,
    Image={
                'S3Object': {
                    'Bucket': s3_image_bucket,
                    'Name':  objectId
                }
            }
        )
    if ret['ResponseMetadata']['HTTPStatusCode'] == 200:
        return ret['FaceRecords'][0]['Face']['FaceId']
    else:
        return False
    

# function to populate faces into index
def populate_index(all_faces,rekognition_face_index):
    for f in all_faces:
        ret = populate_rekognition_collection(f['objectId'])
        if ret == False:
            print ('some error')
        else:
            ret = populate_lookup_table(f['actorFullName'],ret)
            pprint (ret['ResponseMetadata']['HTTPStatusCode'])
        
              
    print ('done')



    
rekognition_face_index = 'actors' 
populate_index(faces,rekognition_face_index)


In [ ]:

sourceurl = 'http://www.presseportal.de/pm/8337/3336775'
source = 'obs/Amazon.de/www.stephan-rabold.com'
imgurl = 'http://cache.pressmailing.net/thumbnail/story_big/eb0a2593-b7a8-4f3f-8b31-5581004c3426/drehstart-fuer-you-are-wanted-der-ersten-deutschen-amazon-originals-serie-von-und-mit-matthias-schwe'

bucket = 'mlrekognitiondemo'
key1 = 'images/yawcast'

# open image
s3.download_file(bucket, key1, 'tmp/image')
image_bin=Image.open('tmp/image')

# resize image for display
image_out=Image.open('tmp/image')
size = 900,600
image_out.thumbnail(size, Image.ANTIALIAS)

# add resized image to image to stream and file for later processing
stream = io.BytesIO()
image_bin.save(stream,format="JPEG")
image_binary = stream.getvalue()
image_bin.save('tmp/image',format="JPEG")

display(image_out)


In [ ]:
# Detect face boxes using Rekognition

def face_detection(image_input):
    response = rekognition.detect_faces(
        Image={
            'Bytes':image_input
        }                                        
        )
    all_faces=response['FaceDetails']
    
    # initialise list object 
    boxes = []
  
    # populate list for each face
    for face in all_faces:
        boxes.append (face['BoundingBox'])
    return boxes

boxes = face_detection(image_binary)
pprint (boxes)

In [ ]:
# And highlight them on the image

image_out = Image.open("tmp/image")
draw = ImageDraw.Draw(image_out)

# get image size
width, height = image_out.size
 
# apply face boxes to image and store coordinates for later processing
def get_coordinates(face_boxes, image_width, image_height):
    # initialise list object 
    coordinates = [] 
    for box in face_boxes:
        x1 = int(box['Left'] * image_width)-5
        y1 = int(box['Top'] * image_height)-5
        x2 = int(box['Left'] * image_width + box['Width'] * image_width)+5
        y2 = int(box['Top'] * image_height + box['Height']  * image_height)+5
        if x1 < 0 : x1=0
        if y1 < 0 : y1=0
        if x2 < 0 : x2=image_width
        if y2 < 0 : y2=image_height 
            
        coordinates.append((x1,y1,x2,y2))
        
    return coordinates
    

coordinates = get_coordinates(boxes,width,height)
    
col = 0
line= 5     
for xy in coordinates: 
    for l in range(line):
        draw.rectangle((xy[0]-l,xy[1]-l,xy[2]+l,xy[3]+l),outline=colors[col])
    col += 1
        
size = 900,600
image_out.thumbnail(size, Image.ANTIALIAS)        

display(image_out)



In [ ]:
# Search faces in index

def get_face_data(image_binary):
    
    try:
        response = rekognition.search_faces_by_image(
            CollectionId='actors',
            Image={
                'Bytes':image_binary
            }                                       
            )
    
        if len(response['FaceMatches']) > 0:
            for o in response['FaceMatches']:
                print (o['Face']['FaceId'],o['Face']['Confidence'],get_actor_name(o['Face']['FaceId']))
            return response['FaceMatches'][0]['Face']['FaceId'],response['FaceMatches'][0]['Face']['Confidence']
        else:
            return ('no match detected',0)
    except Exception as e:
        #print (e)
        return ('no face detected',0)
        
    
# initialise list object 
ret = []    

# iterate through list of boxes and detect individual face
for box in coordinates:
        
        image_crop = image_bin.crop(box)
        display (image_crop)
        
        # add cropped image to temporary stream
        stream2 = io.BytesIO() 
        image_crop.save(stream2,format="JPEG")
        image_region_binary = stream2.getvalue()    
        stream2.close()
        
        try:
            # get external ID name of cropped image
            retdata=get_face_data(image_region_binary),box
            ret.append (retdata)            
        except Exception as e:
            print (e, box)
            ret.append ((('no face',0),box))
            pass


In [ ]:
# Output matches on image

font = ImageFont.load_default()
img_out = Image.open("tmp/image")
draw = ImageDraw.Draw(img_out)

font = ImageFont.truetype("DejaVuSans-Bold.ttf", 22)

col = 0
xpos = 0
ypos = 30
line = 5
for f in ret:
    
    actor_name=get_actor_name(f[0][0]) 
    xpos = len(actor_name)*2.5
    draw.text((f[1][0]-xpos,f[1][1]-ypos),actor_name,colors[col],font=font)
    for l in range(line):
        draw.rectangle((f[1][0]-l,f[1][1]-l,f[1][2]+l,f[1][3]+l),outline=colors[col])
    col += 1
    
display(img_out)

## Celebrity detection

Amazon Rekognition can recognize thousands of celebrities in a wide range of categories, such as entertainment and media, sports, business, and politics.

In [ ]:
%%bash

aws rekognition recognize-celebrities \
--image '{"S3Object":{"Bucket":"mlrekognitiondemo","Name":"images/yawcast"}}' --region eu-west-1 \
--query 'CelebrityFaces[*].{Name:Name,Confidence:MatchConfidence,Details:Urls,Face:Face.BoundingBox}' 

## Content moderation

Provides image moderation to suggestive or explicit content that may not be appropriate for your audience based on two category and eight child category labels.

In [ ]:
bucket = 'mlrekognitiondemo'
key = 'images/david-full-front.jpg'
key = 'images/david-pants.jpg'

# retrieve labels from Rekognition service
ret = rekognition.detect_moderation_labels(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name" : key,
        }
    }
)

# Output image and labels
image_location = 'tmp/image'
s3.download_file(bucket, key, image_location)
size = 500,500
image_bin=Image.open(image_location)
image_bin.thumbnail(size, Image.ANTIALIAS)

display(image_bin)

pprint(ret['ModerationLabels'])

https://eu-west-1.console.aws.amazon.com/rekognition/home?region=eu-west-1#/usage-metrics

all the dependencies that need to be installed:

pip3 install boto3

pip3 install pillow
